In [1]:
import numpy as np # linear algebra
import os
from os.path import join
from glob import glob
from wad import Dict
import cv2
from functools import partial 
from pycocotools.coco import COCO 
import json

In [2]:
def filt(tgt,str_list):
        return sorted([*filter(lambda x: tgt in x, str_list)])

In [5]:
def save_annot(subjet_dir,save_dir,cat_dict,name_prefix="annotations_subj"):
    src_dcm=sorted(glob(join(subjet_dir,"**/","*.dcm"),recursive=True))
    src_png=sorted(glob(join(subjet_dir,"**/","*.png"),recursive=True))
    
    src_f=Dict(img=filt("DICOM_anon",src_dcm),msk=filt("NewGT",src_png))
    categories=[dict(id=v,name=k,supercategory="organ") 
                for ii,(k,v) in enumerate(cat_dict.items())]
    if src_f.msk!=[]:
        images=[dict(id=ii,
                     file_name=img[len(DATA_ROOT)+1:],
                     mask_file=msk[len(DATA_ROOT)+1:]
                    )
                for ii,(img,msk) in enumerate(zip(src_f.img,src_f.msk))]
    else:
        images=[dict(id=ii,
                     file_name=img[len(DATA_ROOT)+1:],
                     mask_file=""
                    )
                for ii,img in enumerate(src_f.img)]
    annotations=[]
    jj=0
    for ii,fname in enumerate(src_f.msk):
        segmap=cv2.imread(fname)
        unique_classes=np.unique(segmap)[1:]
        for c in unique_classes:
            annotations.append(dict(
                id=jj,
                image_id=ii,
                category_id=int(c),
                file_name=fname[len(DATA_ROOT)+1:]))
            jj+=1
    info=dict(
        root_dir=DATA_ROOT.split("/")[-1]
    )
    coco_annt=dict(
        info=info,
        images=images,
        annotations=annotations,
        categories=categories)
    save_fname=join(save_dir,f"{name_prefix}{subjet_dir[len(DATA_ROOT)+1:-1]}.json")
    os.makedirs(save_dir,exist_ok=True)
    with open(save_fname, "w") as f:
        json.dump(coco_annt, f)
    return save_fname

## Save MRI Annotations

In [16]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/MRI/MRI_Label"
kwargs=dict(
    save_dir="../datas/2_Domain_Adaptation_dataset/MRI/annotations/MRI_Label",
    name_prefix="annotations_subj",
    cat_dict=dict(
        trunk=32,
        liver=63,
        kidney_r=126,
        kidney_l=189,
        spleen=252)
)
subjects=sorted(glob(join(DATA_ROOT,"[0-9]*/")))
save_fnames=[*map(partial(save_annot,**kwargs),subjects)]

In [6]:
cc=COCO(save_fnames[0])

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [7]:
cc.loadAnns(cc.getAnnIds(catIds=63))

[{'id': 13,
  'image_id': 12,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00013.png'},
 {'id': 16,
  'image_id': 13,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00014.png'},
 {'id': 19,
  'image_id': 14,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00015.png'},
 {'id': 23,
  'image_id': 15,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00016.png'},
 {'id': 27,
  'image_id': 16,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00017.png'},
 {'id': 31,
  'image_id': 17,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00018.png'},
 {'id': 35,
  'image_id': 18,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00019.png'},
 {'id': 40,
  'image_id': 19,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00020.png'},
 {'id': 45,
  'image_id': 20,
  'category_id': 63,
  'file_name': '1/T2SPIR/NewGT/IMG-0002-00021.png'},
 {'id': 50,
  'image_id': 21,
  'category_id': 63,
  'file_name'

In [17]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/MRI/MRI_nonLabel"
kwargs=dict(kwargs,save_dir="../datas/2_Domain_Adaptation_dataset/MRI/annotations/MRI_nonLabel")
subjects=sorted(glob(join(DATA_ROOT,f"[0-9]*/"),recursive=True))
save_fnames=[*map(partial(save_annot,**kwargs),subjects)]

## Save CT Annotations

In [11]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/CT"
kwargs=dict(
    save_dir="../datas/2_Domain_Adaptation_dataset/CT/annotations",
    cat_dict=dict(
        trunk=32,
        liver=63)
)
subjects=sorted(glob(join(DATA_ROOT,"[0-9]*/")))
save_fnames=[*map(partial(save_annot,**kwargs),subjects)]

## All Subjects

In [18]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/MRI/MRI_Label"
kwargs=dict(
    save_dir="../datas/2_Domain_Adaptation_dataset/MRI/annotations/MRI_Label",
    name_prefix="annotations",
    cat_dict=dict(
        trunk=32,
        liver=63,
        kidney_r=126,
        kidney_l=189,
        spleen=252)
)
fname=save_annot(DATA_ROOT,**kwargs)

In [19]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/MRI/MRI_nonLabel"
kwargs=dict(kwargs,
    save_dir="../datas/2_Domain_Adaptation_dataset/MRI/annotations/MRI_nonLabel",
)
fname=save_annot(DATA_ROOT,**kwargs)

In [5]:
DATA_ROOT="../datas/2_Domain_Adaptation_dataset/CT"
kwargs=dict(kwargs,
    save_dir="../datas/2_Domain_Adaptation_dataset/CT/annotations",
    cat_dict=dict(
        trunk=32,
        liver=63)
)
fname=save_annot(DATA_ROOT,**kwargs)

## Test Set

In [6]:
DATA_ROOT="../UnetCosLoss/data/testset"
kwargs=dict(
    save_dir="../UnetCosLoss/data/testset/annotations",
    name_prefix="annotations_gt",
    cat_dict=dict(
        trunk=32,
        liver=63,
        kidney_r=126,
        kidney_l=189,
        spleen=252)
)
fname=save_annot(DATA_ROOT,**kwargs)